In [20]:
# Importieren der notwendigen Bibliotheken
import numpy as np
from pymongo import MongoClient
from PIL import Image
from io import BytesIO
import base64
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import os

# Verbindung zu MongoDB herstellen
client = MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Funktion zum Dekodieren von Base64-Bildern
def decode_base64_image(base64_string):
    if "," in base64_string:
        base64_data = base64_string.split(",")[1]
    else:
        base64_data = base64_string
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    image = Image.open(image_data).convert("RGB")  # Konvertieren zu RGB
    image = image.resize((75, 75))  # Größe anpassen
    return image

# Funktion zum Vorbereiten des Bildes für das Modell
def prepare_image(image, target_size):
    image_resized = image.resize(target_size)  # Bild auf die Zielgröße ändern
    image_array_resized = np.array(image_resized)
    image_array_normalized = image_array_resized / 255.0
    image_array_normalized = np.expand_dims(image_array_normalized, axis=0)
    return image_array_normalized

# Funktion zum Laden der Modelle basierend auf Präfixen
def load_models(model_dir, prefixes):
    model_paths = [os.path.join(model_dir, f) for f in os.listdir(model_dir) if any(f.startswith(prefix) for prefix in prefixes)]
    model_names = [os.path.basename(path) for path in model_paths]
    models = []
    for path in model_paths:
        if os.path.exists(path):
            models.append(tf.keras.models.load_model(path, compile=False))
        else:
            print(f"Model file not found: {path}")
    for model in models:
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return models, model_names

# Zufälligen Fingerabdruck aus der Datenbank laden
def get_random_fingerprint(username):
    fingerprints = list(collection.aggregate([{"$match": {"username": username}}, {"$sample": {"size": 1}}]))
    if not fingerprints:
        raise ValueError(f"No fingerprints found in the database for username: {username}")
    random_fingerprint = fingerprints[0]
    if 'canvases' not in random_fingerprint or not random_fingerprint['canvases']:
        raise KeyError("The field 'canvases' was not found in the document or is empty")
    base64_image = random.choice(random_fingerprint['canvases'])  # Zufälliges Bild aus dem 'canvases'-Array auswählen
    return decode_base64_image(base64_image)

# Mehrere Fingerabdrücke abrufen und vorbereiten
def get_multiple_fingerprints(usernames, num_tests):
    images = []
    for _ in range(num_tests):
        username = random.choice(usernames)
        print(f"Loading random canvas for username: {username}")
        image = get_random_fingerprint(username)
        images.append((username, image))
    return images

# Präfixe für die Modelle
prefixes = ["username_1", "username_4"]  # Beispiel: kann auch andere Präfixe enthalten

# Anzahl der Tests
num_tests = 10

# Modelle laden und testen
for prefix in prefixes:
    print(f"\nTesting models with prefix: {prefix}")
    models, model_names = load_models('models', [prefix])
    
    # Geladene Modelle anzeigen
    print(f"Loaded models: {model_names}")
    
    # Mehrere Fingerabdrücke abrufen und vorbereiten
    images = get_multiple_fingerprints(prefixes, num_tests)
    prepared_images = [(username, prepare_image(image, target_size=(75, 75))) for username, image in images]
    
    # Ergebnisse speichern
    all_results = {name: [] for name in model_names}
    for username, image_array_normalized in prepared_images:
        for model, name in zip(models, model_names):
            prediction = model.predict(image_array_normalized)
            all_results[name].append((username, prediction[0][0]))
    
    # Ergebnisse in der Konsole ausgeben
    for name in model_names:
        print(f"\nResults for model: {name}")
        for i, (username, result) in enumerate(all_results[name]):
            print(f"Test {i+1}: Username = {username}, Prediction = {result}")
    
    # Ergebnisse grafisch darstellen
    fig, ax = plt.subplots()
    for name in model_names:
        results = [result for username, result in all_results[name]]
        ax.plot(range(num_tests), results, label=name)
    ax.set_xlabel('Test Number')
    ax.set_ylabel('Prediction')
    ax.set_title(f'Model Predictions for Random Fingerprints ({prefix})')
    ax.legend()
    plt.show()

# Fingerabdrücke anzeigen
fig, axes = plt.subplots(1, num_tests, figsize=(20, 5))
for i, (username, image) in enumerate(images):
    axes[i].imshow(image, cmap='gray')
    axes[i].set_title(username)
    axes[i].axis('off')
plt.show()


Testing models with prefix: username_1
Loaded models: ['username_1_1.h5', 'username_1_2.h5', 'username_1_3.h5', 'username_1_4.h5', 'username_1_5.h5', 'username_1_6.h5']
Loading random canvas for username: username_4
Loading random canvas for username: username_4
Loading random canvas for username: username_1
Loading random canvas for username: username_4
Loading random canvas for username: username_1
Loading random canvas for username: username_4
Loading random canvas for username: username_4
Loading random canvas for username: username_4
Loading random canvas for username: username_1
Loading random canvas for username: username_1
1/1 [==============================] - 0s 62ms/step


In [ ]:
# Importieren der notwendigen Bibliotheken
import numpy as np
from pymongo import MongoClient
from PIL import Image
from io import BytesIO
import base64
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import os

# Verbindung zu MongoDB herstellen
client = MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Funktion zum Dekodieren von Base64-Bildern
def decode_base64_image(base64_string):
    if "," in base64_string:
        base64_data = base64_string.split(",")[1]
    else:
        base64_data = base64_string
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    image = Image.open(image_data).convert("RGB")  # Konvertieren zu RGB
    image = image.resize((75, 75))  # Größe anpassen
    return image

# Funktion zum Vorbereiten des Bildes für das Modell
def prepare_image(image, target_size):
    image_resized = image.resize(target_size)  # Bild auf die Zielgröße ändern
    image_array_resized = np.array(image_resized)
    image_array_normalized = image_array_resized / 255.0
    image_array_normalized = np.expand_dims(image_array_normalized, axis=0)
    return image_array_normalized

# Funktion zum Laden der Modelle basierend auf Präfixen
def load_models(model_dir, prefixes):
    model_paths = [os.path.join(model_dir, f) for f in os.listdir(model_dir) if any(f.startswith(prefix) for prefix in prefixes)]
    model_names = [os.path.basename(path) for path in model_paths]
    models = []
    for path in model_paths:
        if os.path.exists(path):
            models.append(tf.keras.models.load_model(path, compile=False))
        else:
            print(f"Model file not found: {path}")
    for model in models:
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return models, model_names

# Zufälligen Fingerabdruck aus der Datenbank laden
def get_random_fingerprint(username):
    fingerprints = list(collection.aggregate([{"$match": {"username": username}}, {"$sample": {"size": 1}}]))
    if not fingerprints:
        raise ValueError(f"No fingerprints found in the database for username: {username}")
    random_fingerprint = fingerprints[0]
    if 'canvases' not in random_fingerprint or not random_fingerprint['canvases']:
        raise KeyError("The field 'canvases' was not found in the document or is empty")
    base64_image = random.choice(random_fingerprint['canvases'])  # Zufälliges Bild aus dem 'canvases'-Array auswählen
    return decode_base64_image(base64_image)

# Mehrere Fingerabdrücke abrufen und vorbereiten
def get_multiple_fingerprints(usernames, num_tests):
    images = []
    for _ in range(num_tests):
        username = random.choice(usernames)
        print(f"Loading random canvas for username: {username}")
        image = get_random_fingerprint(username)
        images.append((username, image))
    return images

# Präfixe für die Modelle
prefixes = ["username_1", "username_4"]  # Beispiel: kann auch andere Präfixe enthalten

# Anzahl der Tests
num_tests = 10

# Modelle laden und testen
for prefix in prefixes:
    print(f"\nTesting models with prefix: {prefix}")
    models, model_names = load_models('models', [prefix])
    
    # Geladene Modelle anzeigen
    print(f"Loaded models: {model_names}")
    
    # Mehrere Fingerabdrücke abrufen und vorbereiten
    images = get_multiple_fingerprints(prefixes, num_tests)
    prepared_images = [(username, prepare_image(image, target_size=(75, 75))) for username, image in images]
    
    # Ergebnisse speichern
    all_results = {name: [] for name in model_names}
    for username, image_array_normalized in prepared_images:
        for model, name in zip(models, model_names):
            prediction = model.predict(image_array_normalized)
            all_results[name].append((username, prediction[0][0]))
    
    # Ergebnisse in der Konsole ausgeben
    for name in model_names:
        print(f"\nResults for model: {name}")
        for i, (username, result) in enumerate(all_results[name]):
            print(f"Test {i+1}: Username = {username}, Prediction = {result}")
    
    # Ergebnisse grafisch darstellen
    fig, ax = plt.subplots()
    for name in model_names:
        results = [result for username, result in all_results[name]]
        ax.plot(range(num_tests), results, label=name)
    ax.set_xlabel('Test Number')
    ax.set_ylabel('Prediction')
    ax.set_title(f'Model Predictions for Random Fingerprints ({prefix})')
    ax.legend()
    plt.show()

# Fingerabdrücke anzeigen
fig, axes = plt.subplots(1, num_tests, figsize=(20, 5))
for i, (username, image) in enumerate(images):
    axes[i].imshow(image, cmap='gray')
    axes[i].set_title(username)
    axes[i].axis('off')
plt.show()

# Zusätzlicher Code zum Testen von username_1 Modellen gegen username_2 Fingerabdrücke und umgekehrt
def test_cross_models(prefix1, prefix2, num_tests):
    print(f"\nTesting {prefix1} models with {prefix2} fingerprints")
    models, model_names = load_models('models', [prefix1])
    
    # Geladene Modelle anzeigen
    print(f"Loaded models: {model_names}")
    
    # Fingerabdrücke von prefix2 abrufen und vorbereiten
    images = get_multiple_fingerprints([prefix2], num_tests)
    prepared_images = [(prefix2, prepare_image(image, target_size=(75, 75))) for _, image in images]
    
    # Ergebnisse speichern
    all_results = {name: [] for name in model_names}
    for _, image_array_normalized in prepared_images:
        for model, name in zip(models, model_names):
            prediction = model.predict(image_array_normalized)
            all_results[name].append((prefix2, prediction[0][0]))
    
    # Ergebnisse in der Konsole ausgeben
    for name in model_names:
        print(f"\nResults for model: {name}")
        for i, (username, result) in enumerate(all_results[name]):
            print(f"Test {i+1}: Username = {username}, Prediction = {result}")
    
    # Ergebnisse grafisch darstellen
    fig, ax = plt.subplots()
    for name in model_names:
        results = [result for username, result in all_results[name]]
        ax.plot(range(num_tests), results, label=name)
    ax.set_xlabel('Test Number')
    ax.set_ylabel('Prediction')
    ax.set_title(f'Model Predictions for {prefix2} Fingerprints ({prefix1} Models)')
    ax.legend()
    plt.show()

# Testen von username_1 Modellen gegen username_2 Fingerabdrücke
test_cross_models("username_1", "username_1", num_tests)

# Testen von username_2 Modellen gegen username_1 Fingerabdrücke
test_cross_models("username_4", "username_4", num_tests)

In [ ]:
# Importieren der notwendigen Bibliotheken
import numpy as np
from pymongo import MongoClient
from PIL import Image
from io import BytesIO
import base64
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Verbindung zu MongoDB herstellen
client = MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Funktion zum Dekodieren von Base64-Bildern
def decode_base64_image(base64_string):
    if "," in base64_string:
        base64_data = base64_string.split(",")[1]
    else:
        base64_data = base64_string
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    image = Image.open(image_data).convert("RGB")  # Konvertieren zu RGB
    image = image.resize((75, 75))  # Größe anpassen
    return image

# Funktion zum Vorbereiten des Bildes für das Modell
def prepare_image(image, target_size):
    image_resized = image.resize(target_size)  # Bild auf die Zielgröße ändern
    image_array_resized = np.array(image_resized)
    image_array_normalized = image_array_resized / 255.0
    image_array_normalized = np.expand_dims(image_array_normalized, axis=0)
    return image_array_normalized

# Funktion zum Laden der Modelle basierend auf Präfixen
def load_models(model_dir, prefixes):
    model_paths = [os.path.join(model_dir, f) for f in os.listdir(model_dir) if any(f.startswith(prefix) for prefix in prefixes)]
    model_names = [os.path.basename(path) for path in model_paths]
    models = []
    for path in model_paths:
        if os.path.exists(path):
            models.append(tf.keras.models.load_model(path, compile=False))
        else:
            print(f"Model file not found: {path}")
    for model in models:
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return models, model_names

# Zufälligen Fingerabdruck aus der Datenbank laden
def get_random_fingerprint(username):
    fingerprints = list(collection.aggregate([{"$match": {"username": username}}, {"$sample": {"size": 1}}]))
    if not fingerprints:
        raise ValueError(f"No fingerprints found in the database for username: {username}")
    random_fingerprint = fingerprints[0]
    if 'canvases' not in random_fingerprint or not random_fingerprint['canvases']:
        raise KeyError("The field 'canvases' was not found in the document or is empty")
    base64_image = random.choice(random_fingerprint['canvases'])  # Zufälliges Bild aus dem 'canvases'-Array auswählen
    return decode_base64_image(base64_image)

# Mehrere Fingerabdrücke abrufen und vorbereiten
def get_multiple_fingerprints(usernames, num_tests):
    images = []
    for _ in range(num_tests):
        username = random.choice(usernames)
        print(f"Loading random canvas for username: {username}")
        image = get_random_fingerprint(username)
        images.append((username, image))
    return images

# Präfixe für die Modelle
prefixes = ["username_1", "username_4"]  # Beispiel: kann auch andere Präfixe enthalten

# Anzahl der Tests
num_tests = 500

# Funktion zur Berechnung der Metriken
def evaluate_model(model, images, true_labels):
    predictions = []
    for _, image_array_normalized in images:
        prediction = model.predict(image_array_normalized)
        predictions.append(prediction[0][0])
    
    predictions = np.array(predictions) > 0.5  # Schwellenwert für binäre Klassifikation
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    conf_matrix = confusion_matrix(true_labels, predictions)
    
    return accuracy, precision, recall, f1, conf_matrix

# Modelle laden und evaluieren
all_results = []
for prefix in prefixes:
    print(f"\nEvaluating models with prefix: {prefix}")
    models, model_names = load_models('models', [prefix])
    
    # Geladene Modelle anzeigen
    print(f"Loaded models: {model_names}")
    
    # Mehrere Fingerabdrücke abrufen und vorbereiten
    images = get_multiple_fingerprints(prefixes, num_tests)
    prepared_images = [(username, prepare_image(image, target_size=(75, 75))) for username, image in images]
    
    # Wahre Labels erstellen (1 für username_1, 0 für username_4)
    true_labels = [1 if username == "username_1" else 0 for username, _ in prepared_images]
    
    # Ergebnisse speichern und Metriken berechnen
    for model, name in zip(models, model_names):
        accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, prepared_images, true_labels)
        all_results.append((name, accuracy, precision, recall, f1, conf_matrix))
        print(f"\nResults for model: {name}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{conf_matrix}")
        
        # Konfusionsmatrix grafisch darstellen
        disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
        disp.plot(cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix for {name}')
        plt.show()

# Zusammenfassung der Ergebnisse
print("\nSummary of Results:")
for name, accuracy, precision, recall, f1, conf_matrix in all_results:
    print(f"\nModel: {name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")

In [ ]:
# Importieren der notwendigen Bibliotheken
import numpy as np
from pymongo import MongoClient
from PIL import Image
from io import BytesIO
import base64
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

# Verbindung zu MongoDB herstellen
client = MongoClient("mongodb://localhost:27017/")
db = client["fingerprintDB"]
collection = db["fingerprints"]

# Funktion zum Dekodieren von Base64-Bildern
def decode_base64_image(base64_string):
    if "," in base64_string:
        base64_data = base64_string.split(",")[1]
    else:
        base64_data = base64_string
    byte_data = base64.b64decode(base64_data)
    image_data = BytesIO(byte_data)
    image = Image.open(image_data).convert("RGB")  # Konvertieren zu RGB
    image = image.resize((75, 75))  # Größe anpassen
    return image

# Funktion zum Vorbereiten des Bildes für das Modell
def prepare_image(image, target_size):
    image_resized = image.resize(target_size)  # Bild auf die Zielgröße ändern
    image_array_resized = np.array(image_resized)
    image_array_normalized = image_array_resized / 255.0
    image_array_normalized = np.expand_dims(image_array_normalized, axis=0)
    return image_array_normalized

# Funktion zum Laden der Modelle basierend auf Präfixen
def load_models(model_dir, prefixes):
    model_paths = [os.path.join(model_dir, f) for f in os.listdir(model_dir) if any(f.startswith(prefix) for prefix in prefixes)]
    model_names = [os.path.basename(path) for path in model_paths]
    models = []
    for path in model_paths:
        if os.path.exists(path):
            models.append(tf.keras.models.load_model(path, compile=False))
        else:
            print(f"Model file not found: {path}")
    for model in models:
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return models, model_names

# Zufälligen Fingerabdruck aus der Datenbank laden
def get_random_fingerprint(username):
    fingerprints = list(collection.aggregate([{"$match": {"username": username}}, {"$sample": {"size": 1}}]))
    if not fingerprints:
        raise ValueError(f"No fingerprints found in the database for username: {username}")
    random_fingerprint = fingerprints[0]
    if 'canvases' not in random_fingerprint or not random_fingerprint['canvases']:
        raise KeyError("The field 'canvases' was not found in the document or is empty")
    base64_image = random.choice(random_fingerprint['canvases'])  # Zufälliges Bild aus dem 'canvases'-Array auswählen
    return decode_base64_image(base64_image)

# Mehrere Fingerabdrücke abrufen und vorbereiten
def get_multiple_fingerprints(usernames, num_tests):
    images = []
    for _ in range(num_tests):
        username = random.choice(usernames)
        print(f"Loading random canvas for username: {username}")
        image = get_random_fingerprint(username)
        images.append((username, image))
    return images

# Präfixe für die Modelle
prefixes = ["username_1", "username_4"]  # Beispiel: kann auch andere Präfixe enthalten

# Anzahl der Tests
num_tests = 500

# Funktion zur Berechnung der Metriken
def evaluate_model(model, images, true_labels):
    predictions = []
    for _, image_array_normalized in images:
        prediction = model.predict(image_array_normalized)
        predictions.append(prediction[0][0])
    
    predictions = np.array(predictions) > 0.5  # Schwellenwert für binäre Klassifikation
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    conf_matrix = confusion_matrix(true_labels, predictions)
    
    return accuracy, precision, recall, f1, conf_matrix

# Modelle laden und evaluieren
all_results = []
for prefix in prefixes:
    print(f"\nEvaluating models with prefix: {prefix}")
    models, model_names = load_models('models', [prefix])
    
    # Geladene Modelle anzeigen
    print(f"Loaded models: {model_names}")
    
    # Mehrere Fingerabdrücke abrufen und vorbereiten
    images = get_multiple_fingerprints(prefixes, num_tests)
    prepared_images = [(username, prepare_image(image, target_size=(75, 75))) for username, image in images]
    
    # Wahre Labels erstellen (1 für username_1, 0 für username_4)
    true_labels = [1 if username == "username_1" else 0 for username, _ in prepared_images]
    
    # Ergebnisse speichern und Metriken berechnen
    for model, name in zip(models, model_names):
        accuracy, precision, recall, f1, conf_matrix = evaluate_model(model, prepared_images, true_labels)
        all_results.append((name, accuracy, precision, recall, f1, conf_matrix))
        print(f"\nResults for model: {name}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{conf_matrix}")
        
        # Konfusionsmatrix grafisch darstellen
        disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
        disp.plot(cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix for {name}')
        plt.show()

# Zusammenfassung der Ergebnisse
print("\nSummary of Results:")
for name, accuracy, precision, recall, f1, conf_matrix in all_results:
    print(f"\nModel: {name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")